Importando as bibliotecas necessárias:

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds



Carregar o Dataset Cats vs Dogs do TensorFlow :



In [ ]:
# Carregar o dataset Cats vs Dogs
dataset_name = "cats_vs_dogs"
(ds_train, ds_val), ds_info = tfds.load(
    dataset_name,
    split=['train[:80%]', 'train[80%:]'],  # 80% para treino e 20% para validação
    with_info=True,
    as_supervised=True,  # Retorna tuplas (imagem, label)
)

# Exibir informações do dataset
print("Informações do dataset:")
print(ds_info)

Pré-Processamento para atender o padrão do modelo ResNet50v2 :

In [9]:
# Função de pré-processamento c
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))  # Redimensionar para 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalizar para [0, 1]
    return image, label

# Carregar novamente o dataset
dataset_name = "cats_vs_dogs"
(ds_train, ds_val), ds_info = tfds.load(
    dataset_name,
    split=['train[:80%]', 'train[80%:]'],  # 80% treino, 20% validação
    as_supervised=True,  # Retorna tuplas (imagem, label)
    with_info=True
)

# Aplicar o pré-processamento
batch_size = 32
ds_train = ds_train.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)


Carregando e Configurando o modelo ResNet50V2 :

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Carregar o modelo base pré-treinado (sem a top layer)
base_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas do modelo base
base_model.trainable = False

# Adicionar camadas finais específicas para gatos e cachorros
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduz a dimensionalidade
x = Dense(128, activation="relu")(x)  # Camada densa com 128 unidades e ReLU
output = Dense(1, activation="sigmoid")(x)  # Camada final binária (sigmoid para 2 classes)

# Criar o modelo final
model = Model(inputs=base_model.input, outputs=output)

# Compilar o modelo
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Resumo do modelo
model.summary()


Treinando o modelo com 5 épocas:

In [ ]:
# Treinamento do modelo
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5  # Ajuste o número de épocas conforme necessário
)


In [ ]:
# Criar uma figura com subplots lado a lado
fig = plt.figure(figsize=(16, 4))

# Plotando a perda de validação
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"], label="Validation Loss")
ax.set_title("Validation Loss")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.legend()

# Plotando a acurácia de validação
ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_accuracy"], label="Validation Accuracy", color="green")
ax2.set_title("Validation Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Accuracy")
ax2.set_ylim(0, 1)
ax2.legend()

# Exibindo os gráficos
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Caminho para a imagem externa
image_path = "./images/aisha.jpg"

# Função para carregar e pré-processar a imagem
def preprocess_image(image_path, target_size=(224, 224)):
    # Carregar a imagem e redimensionar para o tamanho esperado pelo modelo
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)  # Converter para array NumPy
    image_array = image_array / 255.0  # Normalizar para [0, 1]
    return image_array, image

# Pré-processar a imagem
image_array, original_image = preprocess_image(image_path)

# Adicionar dimensão para representar o batch
input_tensor = np.expand_dims(image_array, axis=0)  # (1, 224, 224, 3)

# Fazer a previsão
predicted_prob = model.predict(input_tensor)[0][0]  # Pegar a probabilidade de "gato"

# Determinar a classe com base na probabilidade
predicted_class = "Gato" if predicted_prob > 0.5 else "Cachorro"
confidence = f"{predicted_prob * 100:.2f}%" if predicted_prob > 0.5 else f"{(1 - predicted_prob) * 100:.2f}%"

# Exibir a imagem com o resultado
plt.figure(figsize=(8, 6))
plt.imshow(original_image)
plt.title(f"Predição: {predicted_class} (Confiança: {confidence})", fontsize=16, color="blue")
plt.axis("off")
plt.show()
